In [6]:
!curl -L -o ../data/corona-virus-brazil.zip https://www.kaggle.com/api/v1/datasets/download/unanimad/corona-virus-brazil

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 26.9M  100 26.9M    0     0  9552k      0  0:00:02  0:00:02 --:--:-- 15.5M


In [ ]:
!unzip -o ../data/corona-virus-brazil.zip -d ../data
!rm -rf ../data/corona-virus-brazil.zip

Archive:  ../data/corona-virus-brazil.zip
  inflating: ../data/brazil_cities_coordinates.csv  
  inflating: ../data/brazil_covid19.csv  
  inflating: ../data/brazil_covid19_cities.csv  
  inflating: ../data/brazil_covid19_macro.csv  
  inflating: ../data/brazil_covid19_old.csv  
  inflating: ../data/brazil_population_2019.csv  


In [2]:
import pandas as pd

In [3]:
def ler_dataframe_csv(file):
    df = pd.read_csv(file)
    return df

df = ler_dataframe_csv('../data/brazil_covid19.csv')

df.head()

,date,region,state,cases,deaths
0,2020-02-25,Centro-Oeste,DF,0.0,0
1,2020-02-25,Centro-Oeste,GO,0.0,0
2,2020-02-25,Centro-Oeste,MS,0.0,0
3,2020-02-25,Centro-Oeste,MT,0.0,0
4,2020-02-25,Nordeste,AL,0.0,0


In [8]:
min_date = df['date'].min()
max_date = df['date'].max()

print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

Minimum date: 2020-02-25
Maximum date: 2021-05-23


In [9]:
df_cities = ler_dataframe_csv('../data/brazil_covid19_cities.csv')

df_cities.describe()

,code,cases,deaths
count,2.356110e+06,2.356110e+06,2.356110e+06
mean,3.253586e+05,1.081011e+03,2.961563e+01
std,9.848221e+04,7.849223e+03,3.184565e+02
min,1.100010e+05,0.000000e+00,0.000000e+00
25%,2.512100e+05,2.000000e+01,0.000000e+00
50%,3.146275e+05,1.510000e+02,3.000000e+00
75%,4.119200e+05,5.570000e+02,1.100000e+01
max,5.300100e+05,7.688680e+05,2.986600e+04


In [43]:
# Ler os arquivos CSV
dados_covid_cidades = ler_dataframe_csv('../data/brazil_covid19_cities.csv')
dados_populacao = ler_dataframe_csv('../data/brazil_population_2019.csv')

def cidades_de_alto_impacto(dados_covid_cidades, dados_populacao):
    # Criar um mapeamento de nomes completos dos estados para abreviações
    mapeamento_estados = {
        'Acre': 'AC', 'Alagoas': 'AL', 'Amazonas': 'AM', 'Amapá': 'AP',
        'Bahia': 'BA', 'Ceará': 'CE', 'Distrito Federal': 'DF', 'Espírito Santo': 'ES',
        'Goiás': 'GO', 'Maranhão': 'MA', 'Minas Gerais': 'MG', 'Mato Grosso do Sul': 'MS',
        'Mato Grosso': 'MT', 'Pará': 'PA', 'Paraíba': 'PB', 'Pernambuco': 'PE',
        'Piauí': 'PI', 'Paraná': 'PR', 'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN',
        'Rondônia': 'RO', 'Roraima': 'RR', 'Rio Grande do Sul': 'RS',
        'Santa Catarina': 'SC', 'Sergipe': 'SE', 'São Paulo': 'SP', 'Tocantins': 'TO'
    }
    
    # Renomear a coluna "estado" no DataFrame de população
    dados_populacao['state'] = dados_populacao['state'].replace(mapeamento_estados)
    
    dados_covid_cidades['city'] = dados_covid_cidades['name']
    
    # Converter a coluna "data" para o formato datetime
    dados_covid_cidades['date'] = pd.to_datetime(dados_covid_cidades['date'], format='%Y-%m-%d')
    
    # Realizar o join com os dados de população
    resultado = dados_covid_cidades.merge(dados_populacao, on=['city', 'state'], how='inner')
    
    # Adicionar uma coluna de taxa de mortalidade
    resultado['death_rate'] = resultado['deaths'] / resultado['population']
    
    # Calcular a média móvel de 7 dias para óbitos
    resultado['7_day_avg_deaths'] = (
        resultado.groupby('state')['deaths']
        .rolling(window=7).mean().reset_index(0, drop=True)
    )

    # Ordenar pelos valores de taxa de mortalidade em ordem decrescente
    resultado = resultado.sort_values(by='death_rate', ascending=False)
    return resultado

# Aplicar a função
resultado = cidades_de_alto_impacto(dados_covid_cidades, dados_populacao)

resultado.head()

,date,state,name,code,cases,deaths,city,region,state_code,city_code,health_region_code,health_region,population,death_rate,7_day_avg_deaths
1203440,2021-05-22,PE,Quixaba,261153.0,13784.0,4120,Quixaba,Nordeste,26,261153,26001,Afogados da Ingazeira,6813.0,0.604726,656.428571
1203402,2021-05-22,PE,Itaquitinga,260780.0,4457.0,1409,Itaquitinga,Nordeste,26,260780,26005,Goiana,16955.0,0.083102,222.428571
286603,2020-07-05,MT,Curvelândia,510343.0,22.0,239,Curvelândia,Centro-Oeste,51,510343,51011,Oeste Matogrossense,5219.0,0.045794,35.857143
1203422,2021-05-22,PE,Nazaré da Mata,260950.0,3338.0,830,Nazaré da Mata,Nordeste,26,260950,26006,Limoeiro,32471.0,0.025561,149.000000
1203430,2021-05-22,PE,Paudalho,261060.0,4240.0,672,Paudalho,Nordeste,26,261060,26006,Limoeiro,56506.0,0.011893,138.428571


In [41]:
dados_covid_cidades.loc[(dados_covid_cidades['name'] == 'Quixaba') & (dados_covid_cidades['state'] == 'PE')]

,date,state,name,code,cases,deaths
2915,2020-03-27,PE,Quixaba,261153.0,0.0,0
8485,2020-03-28,PE,Quixaba,261153.0,0.0,0
14055,2020-03-29,PE,Quixaba,261153.0,0.0,0
19625,2020-03-30,PE,Quixaba,261153.0,0.0,0
25195,2020-03-31,PE,Quixaba,261153.0,0.0,0
...,...,...,...,...,...,...
2331175,2021-05-19,PE,Quixaba,261153.0,381.0,11
2336745,2021-05-20,PE,Quixaba,261153.0,382.0,11
2342315,2021-05-21,PE,Quixaba,261153.0,383.0,11
2347885,2021-05-22,PE,Quixaba,261153.0,13784.0,4120
